# An Introduction to Building Pipelines and Using Grid Searches in Scikit-learn

There are several steps to building even a basic machine learning model. Before any new data can be fed to a model, features have to be selected, data needs to be standardized and the exact type estimator to be used have to be selected and fit to training data. On top of all that, building models involve two types of parameters:  
i) **model parameters**: configuration variables internal to the model and which can be estimated from data; and,  
ii) **model hyperparameters**: configuration variables external to the model and cannot be estimated from data.[[1]](https://machinelearningmastery.com/difference-between-a-parameter-and-a-hyperparameter/)  
Hyperparameters must be supplied externally and adjusting them is a large part of developing a model. This process is often also referred to as hyperparameter tuning and involves the data scientist optimizing such parameters for performance. Throughout the model building process there are many steps where it is necessary to provide hyperparameters. 

Building a model is an iterative process. And as one would imagine, the process can easily become tedious, unwieldy and error-prone. Thankfully Scikit-Learn has a great set of tools meant to address exactly this: pipeline and gridsearch. This goal of this article is to demonstrate the usage of these tools. Before we forward it's important to mention, however, that the rest of the article is not a exercise in regression analysis. The analytical work here is simply a vehicle to demonstrate the Scikit learn tools. With that out of the way, let's get started. As always we start by importing the necessary libraries. As for libraries relevant to the subject of this article: we need ```make_pipeline``` from ```sklearn.pipeline``` to create a pipeline. The ```GridSearchCV``` method comes from ```model_selection``` module in Scikit-Learn library. We will use the Penguins dataset that ships with Seaborn.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns;
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import Ridge, Lasso

/home/user-1131/venv/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
penguins=sns.load_dataset('penguins')

In [3]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


To illustrate the process of building a pipeline, I'll use regression to estimate body mass, which is a continuous numeric variable. Let's choose 3 columns with numerical data (```bill length```, ```bill depth```, ```flipper length```) and 1 with categorical variable (```sex```) as features. We don't need to include ```species``` as the other features already encapsulates that information. The ```sex``` column has some null values and we'll drop them.

In [4]:
penguins.dropna(inplace=True)

Let's create a feature matrix (**X**) and a matrix with target variable (**y**). Since ```sex``` is a categorical variable, we'll also need to dummify them before splitting the matrices into training and testing sets. 

In [5]:
X=penguins.drop(['island','body_mass_g'],axis=1)

In [6]:
X[['species','sex']]

species     sex
0    Adelie    MALE
1    Adelie  FEMALE
2    Adelie  FEMALE
4    Adelie  FEMALE
5    Adelie    MALE
..      ...     ...
338  Gentoo  FEMALE
340  Gentoo  FEMALE
341  Gentoo    MALE
342  Gentoo  FEMALE
343  Gentoo    MALE

[333 rows x 2 columns]

In [7]:
#Let's pass 'species' column to OneHotEncoder
X=pd.get_dummies(data=X,columns=['species','sex'],drop_first=True)

In [8]:
y=penguins['body_mass_g']

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [10]:
X_train.head(2)

bill_length_mm  bill_depth_mm  flipper_length_mm  species_Chinstrap  \
285            49.8           16.8              230.0                  0   
29             40.5           18.9              180.0                  0   

     species_Gentoo  sex_MALE  
285               1         1  
29                0         1

We are finally ready to set-up a pipeline and build a model. The ```pipeline``` module in Scikit-Learn has a ```make-pipeline``` method. The first step is to to instantiate the method. We do this by passing the steps we want input data to go through in order. Once instantiated the pipeline works just like any other Scikit-Learn estimator. Here we are building a new pipeline and naming it pipe. The methods specified within ```make_pipeline``` from left to right are:  
i) ```StandardScaler()```-> We have not standardized the data before ```train_test_split```. So incoming data needs to standardized before any other transformation is performed on it.  
ii) ```SelectKBest()``` -> This method comes from ```feature_selection``` module of Scikit-learn. It selects the best features based on a scoring function (in this case, ```f_regression)```. The number of features is specified by the value of parameter ```k```. We want to try different number of features and find what works best in terms of performance. We can do that in the ```GridSearchCV``` process and will come back to it shortly.  
iii)```Ridge()``` -> This is an estimator that performs the actual regression. The name of the method refers to Tikhonov regularization (more commonly known as ridge regression) performed to reduce the effect of multicollinearity. Like the parameter k discussed previously, we want to test several different values for the various parameters of ridge regression. We do that as part of a grid search, as we discuss next.

In [11]:
pipe=make_pipeline(StandardScaler(),SelectKBest(f_regression),Ridge())

Our pipeline is now ready to be fitted. Like previously mentioned, pipeline acts just like any other estimator. It can accept parameters for every method that is part of the pipeline. A quick way to get a list of parameters that a pipeline can accept is show below:

In [12]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardscaler', 'selectkbest', 'ridge', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'selectkbest__k', 'selectkbest__score_func', 'ridge__alpha', 'ridge__copy_X', 'ridge__fit_intercept', 'ridge__max_iter', 'ridge__normalize', 'ridge__random_state', 'ridge__solver', 'ridge__tol'])

Grid search gives us the ability to search over specified values for each of the parameters listed above. We do this by creating a dictionary with names of parameters as keys and lists of parameter settings to try as values. In our example we call this dictionary ```params``` and pass it ```GridSearchCV```. Notice the parameter n_jobs. It tells Scikit-learn how many jobs to run in parallel. Setting to -1 is equivalent to instructing Scikit-learn to use all processors. Once fitted, the ```GridSearchCV``` instance ```gs``` acts just like any other estimator.

In [13]:
params={
    'selectkbest__k':[1,2,3,4,5,6],
    'ridge__fit_intercept':[True,False],
    'ridge__alpha':[0.0001,0.001,0.01,0.1,1,10,100,1000],
    'ridge__solver':[ 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

In [14]:
gs=GridSearchCV(pipe,params,n_jobs=-1,cv=5)  

In [15]:
gs.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('selectkbest',
                                        SelectKBest(score_func=<function f_regression at 0x7f03b0bf2d08>)),
                                       ('ridge', Ridge())]),
             n_jobs=-1,
             param_grid={'ridge__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                                          1000],
                         'ridge__fit_intercept': [True, False],
                         'ridge__solver': ['svd', 'cholesky', 'lsqr',
                                           'sparse_cg', 'sag', 'saga'],
                         'selectkbest__k': [1, 2, 3, 4, 5, 6]})

On top of searching over all the permutations of the selected parameters, GridSearchCV performs cross-validation on training data. The default value is 5 times but we can specify any other number using the parameter ```cv```. The attribute ```cv_results_``` includes detailed results for each cross-validation run and provides a wealth of data that can be used to determine the fit and robustness of the model. Details of the permutation of parameters that performs the best is provided by the ```best_params_``` attribute.

In [16]:
df_cv_scores=pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score')
df_cv_scores[['params','split0_test_score', 'split1_test_score', 'split2_test_score',\
       'split3_test_score', 'split4_test_score', 'mean_test_score',\
       'std_test_score', 'rank_test_score']].head()

params  split0_test_score  \
317  {'ridge__alpha': 1, 'ridge__fit_intercept': Tr...           0.880676   
299  {'ridge__alpha': 1, 'ridge__fit_intercept': Tr...           0.880632   
293  {'ridge__alpha': 1, 'ridge__fit_intercept': Tr...           0.880632   
305  {'ridge__alpha': 1, 'ridge__fit_intercept': Tr...           0.880632   
323  {'ridge__alpha': 1, 'ridge__fit_intercept': Tr...           0.880620   

     split1_test_score  split2_test_score  split3_test_score  \
317           0.803462           0.836905           0.898847   
299           0.803406           0.836768           0.898888   
293           0.803406           0.836768           0.898888   
305           0.803406           0.836768           0.898888   
323           0.803301           0.836758           0.898909   

     split4_test_score  mean_test_score  std_test_score  rank_test_score  
317           0.854736         0.854925        0.033363                1  
299           0.854679         0.854875        0.033399                2  
293           0.854679         0.854875        0.033399                2  
305           0.854679         0.854875        0.033399                4  
323           0.854733         0.854864        0.033436                5

In [17]:
gs.best_params_

{'ridge__alpha': 1,
 'ridge__fit_intercept': True,
 'ridge__solver': 'sag',
 'selectkbest__k': 6}

Finally, we can predict target values from test set by passing it's feature matrix to ```gs```. Comparing predictions with actual target values is a very effective way of visualizing and communicating performance of a model.

In [18]:
gs.score(X_test,y_test)

0.9089492595461265

In [19]:
y_preds=gs.predict(X_test)

In [20]:
plt.scatter(y_test,y_preds);